## **Beginning**

In [ ]:
import requests
f = open("title.txt", "r")
scripts = f.read().splitlines()
f.close()
wiki = []
for script in scripts:
    print(script + " Film")
    url = "https://en.wikipedia.org/w/api.php?action=query&origin=*&format=json&generator=search&gsrnamespace=0&gsrlimit=5&gsrsearch='{}'".format(script + " Film")
    response = requests.get(url)
    data = response.json()
    pages = data['query']['pages']
    for page in pages.keys():
        if pages[page]['index'] == 1:
            print(pages[page]['pageid'])
            newurl = "https://en.wikipedia.org/w/api.php?action=query&prop=info&pageids={}&inprop=url&format=json".format(pages[page]['pageid'])
            newresponse = requests.get(newurl)
            newdata = newresponse.json()
            wiki.append(newdata["query"]["pages"][page]["fullurl"])
print(wiki)

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

script_data = pd.DataFrame(columns=['Title', 'Plot', 'Cast', 'Director', 'Writer', 'Production', 'Runtime', 'Budget', 'Box office', 'Rating', 'Genre'])

def get_movie_details(index):
    try:
        script_data.loc[index] = [np.nan] * 11
        print(scripts[index])
        print(wiki[index])
        script_data['Title'][index] = scripts[index]
        response = requests.get(wiki[index])
        soup = BeautifulSoup(response.text, 'lxml')

        for section in soup.find_all('h2'):
            if("Plot" in section.get_text()):
                plot = ""
                for sibling in section.find_next_siblings():
                    if sibling.name == 'h2':
                        break
                    text = sibling.get_text()
                    if text:
                        while text[0] == '\n':
                            text = text[1:]
                        if text[-1] == "\n":
                            text = text[:-1] + " "
                    plot += text
                script_data['Plot'][index] = plot
            elif("Cast" in section.get_text()):
                cast = {}
                for sibling in section.find_next_siblings():
                    if sibling.name == 'h2':
                        break
                    elif sibling.name == 'style':
                        continue
                    text = sibling.get_text()
                    text = text.split("\n")[1:-1]
                    for i in text:
                        if " as " in i:
                            cast[i.split(" as ")[0]] = i.split(" as ")[1]
                        else:
                            cast[i] = "Unknown"
                script_data['Cast'][index] = cast
        urls = soup.find_all('a', class_ = "external text")
        imdb_url = ''
        for url in urls:
            if "imdb" in str(url):
                imdb_url = url['href']
                break
        table = soup.find('table', class_ = "infobox vevent")
        trs = table.find_all('tr')
        for tr in trs:
            if("Directed by" in tr.get_text()):
                td = tr.find('td').text
                script_data['Director'][index] = td.split('\n')
            elif("Story by" in tr.get_text()):
                td = tr.find('td').text
                script_data['Writer'][index] = td.split('\n')
            elif("Production" in tr.get_text()):
                td = tr.find('td').text
                script_data['Production'][index] = td.split('\n')
            elif("Running" in tr.get_text()):
                td = tr.find('td').text.split('\n')[0]
                if "[1]" in td:
                    td = td[:-3]
                script_data['Runtime'][index] = td
            elif("Budget" in tr.get_text()):
                td = tr.find('td').text
                script_data['Budget'][index] = td[:-3]
            elif("Box office" in tr.get_text()):
                td = tr.find('td').text
                script_data['Box office'][index] = td[:-3]
        req = Request(url=imdb_url,headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'lxml')
        values = eval(soup.find('script', type="application/ld+json").text)
        script_data['Rating'][index] = values['aggregateRating']['ratingValue']
        if(values['genre']):
            script_data['Genre'][index] = values['genre']
    except Exception as e:
        print(e)

for i in range(0, len(wiki)):
    get_movie_details(i)

script_data.to_csv('script_data.csv', index=False)

In [ ]:
script_data = script_data.drop_duplicates(subset=['Title']).reset_index(drop=True)
script_data.to_csv('script_data.csv', index=False)

## **Here**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
import pandas as pd
import ast
import numpy as np
def func1(x):
    try:
        return ast.literal_eval(x)
    except:
        return np.nan
script_data = pd.read_csv('https://docs.google.com/spreadsheets/d/1X21OVNellmT5hlitSp6i942ruHy4TbMBDzRQ-VUSMRg/export?format=csv&gid=1202267698')
script_data['Cast'] = script_data['Cast'].apply(lambda x: func1(x))
script_data['Director'] = script_data['Director'].apply(lambda x: func1(x))
script_data['Writer'] = script_data['Writer'].apply(lambda x: func1(x))
script_data['Production'] = script_data['Production'].apply(lambda x: func1(x))
script_data['Genre'] = script_data['Genre'].apply(lambda x: func1(x))
script_data.head()

,Title,Plot,Cast,Director,Writer,Production,Runtime,Budget,Box office,Rating,Genre
0,Ace Ventura: Pet Detective (1994),Ace Ventura is an unorthodox Miami-based priva...,"{'Jim Carrey': 'Ace Ventura', 'Sean Young': 'L...",[Tom Shadyac],[Jack Bernstein],[Morgan Creek Productions],86 minutes,$15 million,$107.2 million,6.9,[Comedy]
1,Affliction (1997),"Rolfe Whitehouse begins the film, announcing t...","{'Nick Nolte': 'Wade Whitehouse', 'James Cobur...",[Paul Schrader],NaN,"[, Largo Entertainment, Reisman Productions, K...",114 minutes,$6 mill,$6.3 million[2],6.9,"[Drama, Mystery, Thriller]"
2,Agnes Of God (1985),"In a Roman Catholic convent in Montreal, Quebe...",{'Anne Bancroft': 'Mother Miriam Ruth (religio...,[Norman Jewison],NaN,NaN,99 minutes,$7.5 million,$25.6 mill,6.6,"[Drama, Mystery]"
3,Air Force One (1997),A joint operation between American and Russian...,"{'Gary Oldman': 'Egor Korshunov, a ruthless Ra...",[Wolfgang Petersen],NaN,"[, Columbia Pictures, Beacon Pictures, Radiant...",124 minutes,$85 million,$315.2 million,NaN,NaN
4,Airplane! (1980),Ex-fighter pilot Ted Striker is a traumatized ...,"{'Robert Hays': 'Ted Striker', 'Julie Hagerty'...","[, Jim Abrahams, David Zucker, Jerry Zucker, ]",NaN,[Paramount PicturesHoward W. Koch Productions],87 minutes[2],$3.5 million,$171 million,7.7,[Comedy]


In [ ]:
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 10.3 MB/s eta 0:00:00


In [ ]:
script_sample = script_data.sample(10)

In [ ]:
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset
hg_dataset = Dataset(pa.Table.from_pandas(script_sample.reset_index(drop=True)))
script_values = []
for i in range(len(hg_dataset)):
    title = script_data['Title'][i].replace(':', '-').replace('(','- ').replace(')', '')
    f = open("/content/drive/MyDrive/SEM 5/NLP_Project/SFY Scripts/" + title + ".txt", "r", encoding="utf8")
    script = f.read()
    f.close()
    script_values.append(script)
script_sample['Script'] = script_values

In [ ]:
!pip install transformers -q
!pip install accelerate -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.2 MB/s eta 0:00:00


In [ ]:
df = script_sample

In [ ]:
df

,Title,Plot,Cast,Director,Writer,Production,Runtime,Budget,Box office,Rating,Genre,Script
303,Mulholland Dr. (2001),A dark-haired woman is the sole survivor of a ...,"{'Naomi Watts': 'Betty Elms/Diane Selwyn', 'La...",[David Lynch],NaN,"[, Les Films Alain Sarde, Asymmetrical Product...",146 minutes[7],$15 million,$20.1 million,7.9,"[Drama, Mystery, Thriller]",Ace Ventura: Pet Detective (1994)\nby Jack Ber...
245,L.A. Confidential (1997),"In 1953 Los Angeles, LAPD Sergeant Edmund Exle...",{'Russell Crowe': 'Detective Sergeant Wendell ...,[Curtis Hanson],NaN,[Regency EnterprisesThe Wolper Organization],138 minutes,$35 million,$126.2 million,NaN,NaN,Affliction (1997)\nby Paul Schrader.Based on a...
221,JFK (1991),"During his farewell address in 1961, outgoing ...","{'Kevin Costner': 'Jim Garrison', 'Kevin Bacon...",[Oliver Stone],NaN,[Le Studio Canal+Regency EnterprisesAlcor Film...,188 minutes[1]205 minutes (director's c,$40 mill,$205.4 mill,8.0,"[Drama, History, Thriller]",Agnes Of God (1985)\nby John Pielmeier.Based o...
489,Tomorrow Never Dies (1997),MI6 sends James Bond into the field to reconno...,"{'Pierce Brosnan': 'James Bond, MI6 agent 007....",[Roger Spottiswoode],NaN,[Eon ProductionsUnited Artists],119 minutes,$110 mill,$333 million,NaN,NaN,Air Force One (1997)\nby Andrew Marlow.\nFADE ...
58,Birthday Girl (2001),"John Buckingham (Ben Chaplin), an introverted,...","{'Ben Chaplin': 'John Buckingham', 'Vincent Ca...",[Jez Butterworth],NaN,"[, HAL Films, FilmFour Productions, ]",93 minutes,$13 million,$16.2 million,6.0,"[Comedy, Crime, Thriller]","Airplane! (1980)\nby Jim Abrahams, David Zucke..."
411,Sounder (1972),"In 1933, the Morgans are an African-American f...","{'Cicely Tyson': 'Rebecca Morgan', 'Paul Winfi...",[Martin Ritt],NaN,[Radnitz/Mattel Productions],105 minutes,$1.9 million,$16.9 million,7.5,"[Drama, Family]",Alien (1979)\nby Walter Hill and David Giler.B...
490,Top Gun (1986),More than 30 years after graduating from Top G...,"{'Tom Cruise': 'Captain Pete ""Maverick"" Mitche...",[Joseph Kosinski],"[, Peter Craig, Justin Marks, ]","[, Paramount Pictures, Skydance, TC Production...",130 minutes[2],$170–177 million[3],$1.496 billion[4],8.3,"[Action, Drama]",Alien 3 (1992)\nby William Gibson.Revised firs...
215,Innerspace (1987),"In San Francisco, down-on-his-luck U.S. Navy a...","{'Dennis Quaid': 'Lt. Tuck Pendleton', 'Martin...",[Joe Dante],[Chip Proser],[Amblin Entertainment],120 minutes,$27 mill,$42 mill,6.8,"[Action, Adventure, Comedy]",Alien Nation (1988)\nby Rochne O'Bannon.Rewrit...
395,Siege (1998),FBI Special Agent Anthony Hubbard and his Leba...,{'Annette Bening': 'CIA Operative Sharon Bridg...,[Edward Zwick],[Lawrence Wright],[Twin River Productions],116 minutes,$70 mill,$116.7 mill,6.4,"[Action, Thriller]",Alien: Resurrection (1997)\nby Joss Wedon.Fina...
421,Star Wars: Episode I - The Phantom Menace (1999),The Trade Federation upsets order in the Galac...,"{'Ewan McGregor': 'Obi-Wan Kenobi, Qui-Gon's t...",[George Lucas],NaN,[Lucasfilm Ltd.],133 minutes[1],$115 million,$1.027 billion,6.5,"[Action, Adventure, Fantasy]",Aliens (1986)\nby James Cameron.First draft. M...


In [ ]:
df = df.fillna("")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 303 to 421
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       10 non-null     object
 1   Plot        10 non-null     object
 2   Cast        10 non-null     object
 3   Director    10 non-null     object
 4   Writer      10 non-null     object
 5   Production  10 non-null     object
 6   Runtime     10 non-null     object
 7   Budget      10 non-null     object
 8   Box office  10 non-null     object
 9   Rating      10 non-null     object
 10  Genre       10 non-null     object
 11  Script      10 non-null     object
dtypes: object(12)
memory usage: 1.0+ KB


In [ ]:
df['Director'] = df['Director'].apply(lambda x: ", ".join(i for i in x if len(i)>0))
df['Writer'] = df['Writer'].apply(lambda x: ", ".join(i for i in x if len(i)>0))
df['Production'] = df['Production'].apply(lambda x: ", ".join(i for i in x if len(i)>0))
df['Genre'] = df['Genre'].apply(lambda x: ", ".join(i for i in x if len(i)>0))

In [ ]:
df

,Title,Plot,Cast,Director,Writer,Production,Runtime,Budget,Box office,Rating,Genre,Script
303,Mulholland Dr. (2001),A dark-haired woman is the sole survivor of a ...,"{'Naomi Watts': 'Betty Elms/Diane Selwyn', 'La...",David Lynch,,"Les Films Alain Sarde, Asymmetrical Production...",146 minutes[7],$15 million,$20.1 million,7.9,"Drama, Mystery, Thriller",Ace Ventura: Pet Detective (1994)\nby Jack Ber...
245,L.A. Confidential (1997),"In 1953 Los Angeles, LAPD Sergeant Edmund Exle...",{'Russell Crowe': 'Detective Sergeant Wendell ...,Curtis Hanson,,Regency EnterprisesThe Wolper Organization,138 minutes,$35 million,$126.2 million,,,Affliction (1997)\nby Paul Schrader.Based on a...
221,JFK (1991),"During his farewell address in 1961, outgoing ...","{'Kevin Costner': 'Jim Garrison', 'Kevin Bacon...",Oliver Stone,,Le Studio Canal+Regency EnterprisesAlcor Films...,188 minutes[1]205 minutes (director's c,$40 mill,$205.4 mill,8.0,"Drama, History, Thriller",Agnes Of God (1985)\nby John Pielmeier.Based o...
489,Tomorrow Never Dies (1997),MI6 sends James Bond into the field to reconno...,"{'Pierce Brosnan': 'James Bond, MI6 agent 007....",Roger Spottiswoode,,Eon ProductionsUnited Artists,119 minutes,$110 mill,$333 million,,,Air Force One (1997)\nby Andrew Marlow.\nFADE ...
58,Birthday Girl (2001),"John Buckingham (Ben Chaplin), an introverted,...","{'Ben Chaplin': 'John Buckingham', 'Vincent Ca...",Jez Butterworth,,"HAL Films, FilmFour Productions",93 minutes,$13 million,$16.2 million,6.0,"Comedy, Crime, Thriller","Airplane! (1980)\nby Jim Abrahams, David Zucke..."
411,Sounder (1972),"In 1933, the Morgans are an African-American f...","{'Cicely Tyson': 'Rebecca Morgan', 'Paul Winfi...",Martin Ritt,,Radnitz/Mattel Productions,105 minutes,$1.9 million,$16.9 million,7.5,"Drama, Family",Alien (1979)\nby Walter Hill and David Giler.B...
490,Top Gun (1986),More than 30 years after graduating from Top G...,"{'Tom Cruise': 'Captain Pete ""Maverick"" Mitche...",Joseph Kosinski,"Peter Craig, Justin Marks","Paramount Pictures, Skydance, TC Productions, ...",130 minutes[2],$170–177 million[3],$1.496 billion[4],8.3,"Action, Drama",Alien 3 (1992)\nby William Gibson.Revised firs...
215,Innerspace (1987),"In San Francisco, down-on-his-luck U.S. Navy a...","{'Dennis Quaid': 'Lt. Tuck Pendleton', 'Martin...",Joe Dante,Chip Proser,Amblin Entertainment,120 minutes,$27 mill,$42 mill,6.8,"Action, Adventure, Comedy",Alien Nation (1988)\nby Rochne O'Bannon.Rewrit...
395,Siege (1998),FBI Special Agent Anthony Hubbard and his Leba...,{'Annette Bening': 'CIA Operative Sharon Bridg...,Edward Zwick,Lawrence Wright,Twin River Productions,116 minutes,$70 mill,$116.7 mill,6.4,"Action, Thriller",Alien: Resurrection (1997)\nby Joss Wedon.Fina...
421,Star Wars: Episode I - The Phantom Menace (1999),The Trade Federation upsets order in the Galac...,"{'Ewan McGregor': 'Obi-Wan Kenobi, Qui-Gon's t...",George Lucas,,Lucasfilm Ltd.,133 minutes[1],$115 million,$1.027 billion,6.5,"Action, Adventure, Fantasy",Aliens (1986)\nby James Cameron.First draft. M...


In [ ]:
df.drop('Cast', axis = 1, inplace = True)

In [ ]:
contexts = []
input_ids = []
for index, row in df.iterrows():
    input_id = str(index)
    # Combine the features into a single context string
    context = f"Title: {row['Title']}\n" \
              f"Plot: {row['Plot']}\n" \
              f"Director: {row['Director']}\n" \
              f"Writer: {row['Writer']}\n" \
              f"Production: {row['Production']}\n" \
              f"Runtime: {row['Runtime']}\n" \
              f"Budget: {row['Budget']}\n" \
              f"Box Office: {row['Box office']}\n" \
              f"Rating: {row['Rating']}\n" \
              f"Genre: {row['Genre']}" \
              f"Script:{row['Script']}"
    contexts.append(context)
    input_ids.append(input_id)

In [ ]:
data = {'input_id':input_ids, 'context':contexts}
# Convert the dictionary into a Hugging Face Dataset
dataset = Dataset.from_dict(data)

In [ ]:
dataset = dataset.train_test_split(test_size=0.3)

In [ ]:
dataset['train'][0]

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, TrainingArguments, Trainer
# Initialize the model and tokenizer
model_name = 'distilgpt2'  # You can choose a different model
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

(…)e.co/distilgpt2/resolve/main/config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

(…)gpt2/resolve/main/generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

(…)ce.co/distilgpt2/resolve/main/vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

(…)ce.co/distilgpt2/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)o/distilgpt2/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["context"]])

In [ ]:
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/7 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (68039 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (108622 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (197425 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (213732 > 1024). Running this sequence through the model will result in indexing errors
num_proc must be <= 3. Reducing num_proc to 3 for dataset of size 3.


Map (num_proc=3):   0%|          | 0/3 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (125900 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (122933 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (175490 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
block_size = 256

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/7 [00:00<?, ? examples/s]

num_proc must be <= 3. Reducing num_proc to 3 for dataset of size 3.


Map (num_proc=3):   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
lm_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4285
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1709
    })
})

In [ ]:
import pickle
pickle.dump(lm_dataset, open('/content/drive/MyDrive/SEM 5/NLP_Project/lm_dataset.pkl', 'wb'))

In [ ]:
import pickle
lm_dataset = pickle.load(open('/content/drive/MyDrive/SEM 5/NLP_Project/lm_dataset.pkl', 'rb'))

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
training_args = TrainingArguments(
    output_dir="moviescript",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
# trainer.push_to_hub()

In [ ]:
prompt = "A boy loses his parents in a chaos then become a batman of Gotham City"

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="moviescript")
generator(prompt)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("moviescript")
inputs = tokenizer(prompt, return_tensors="pt").input_ids

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("moviescript")
outputs = model.generate(inputs, max_new_tokens=1000, do_sample=True, top_k=50, top_p=0.95)

In [ ]:
tokenizer.batch_decode(outputs, skip_special_tokens=True)